In [35]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

with open("input_20.txt", 'r') as f:
    data = f.read().rstrip()

#print(colors, designs)


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]

#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]

#data = [(int(row.split(",")[0]),int(row.split(",")[1])) for row in data.split("\n")]

#grid, moves = data.split("\n\n")
#moves = moves.replace("\n", "")

grid = {(x,y): str(val) for y, r in enumerate(data.split('\n')) for x, val in enumerate(r)}
#grid
#data = [int(x) for x in data.split(" ")]


# Output the parsed data
#print(data)


#grid = {(x,y): "." for x in range(max_coor+1) for y in range(max_coor+1)}

#helpers.visualize_dict_grid(grid) 

In [36]:
import heapq

def heuristic(a, b):
    """Calculate Manhattan distance as the heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, goal):
    """A* algorithm to find the shortest path in a grid."""
    if grid[start] == '#' or grid[goal] == '#':
        return None  # Start or goal is blocked

    # Priority queue for the open set
    open_set = []
    heapq.heappush(open_set, (0, start))

    # Came from map for reconstructing the path
    came_from = {}

    # Cost from start to current node
    g_score = {node: float('inf') for node in grid}
    g_score[start] = 0

    # Total cost of the current node
    f_score = {node: float('inf') for node in grid}
    f_score[start] = heuristic(start, goal)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        # Check neighbors
        neighbors = [
            (current[0] + dx, current[1] + dy)
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
        ]
        for neighbor in neighbors:
            if neighbor not in grid or grid[neighbor] == '#':
                continue

            tentative_g_score = g_score[current] + 1  # Assume uniform cost grid

            if tentative_g_score < g_score[neighbor]:
                # This path to neighbor is better
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                if neighbor not in [item[1] for item in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No path found

In [37]:
from collections import Counter

start = [k for k,v in grid.items() if v == "S"][0]
goal = [k for k,v in grid.items() if v == "E"][0]
golden_path = astar(grid, start, goal)
golden_path_len = len(golden_path)

times = {}

for i, x in enumerate(reversed(golden_path)):
    times[x] = (golden_path_len-i-1, i) #(s moved thus far, s until goal)

def reachable_states_with_moves(start, max_moves):
    x_start, y_start = start
    reachable = set()
    
    # Generate all possible moves within the Manhattan distance
    for dx in range(-max_moves, max_moves + 1):
        for dy in range(-max_moves, max_moves + 1):
            moves_taken = abs(dx) + abs(dy)
            if moves_taken <= max_moves and moves_taken > 1:
                reachable.add(((x_start + dx, y_start + dy), moves_taken))
    
    return reachable


max_moves = 20 #using 2 will give you the answer for pt 1 

cheat_lengths = []

# Output the results
#print(f"Number of reachable states: {len(reachable_with_moves)}")
for node in golden_path[:-1]:
    reachable_with_moves = reachable_states_with_moves(node, max_moves)
    for state, moves in sorted(reachable_with_moves, key=lambda x: x[1]):  # Sort by moves_taken
        if state in grid.keys():
            if grid[state] in {".", "E", "S"}:
                #print(f"State: {state}, Moves: {moves}")
                time_saved = golden_path_len-1 - (times[node][0] + times[state][1] + moves)
                if time_saved > 0:
                    cheat_lengths.append(time_saved)
                    
cheat_counts = dict(sorted(Counter(cheat_lengths).items(), key=lambda x: x[0]))
ans = 0
for k,v in cheat_counts.items():
    if k >= 100:
        ans += v
ans

990096